## Import Statements

In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import csv
import re
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
import pandas as pd





[nltk_data] Downloading package stopwords to /Users/sulu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sulu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load and save the model from the internet

In [8]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

2023-09-16 12:02:03.997845: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


: 

: 

## Preprocess the text

In [6]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(lemmatized_words)
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

## Test the model on validation Data

In [5]:
import pandas as pd

In [10]:
# Laden Sie Ihren CSV-Datensatz
csv_file = "../../data/val_data.csv"  # Ersetzen Sie durch den Pfad zu Ihrer CSV-Datei
df = pd.read_csv(csv_file)

# Leere Listen zum Speichern der vorhergesagten Sentiments und der tatsächlichen Labels
predicted_sentiments = []
actual_labels = []

# Durchlaufen Sie jeden Eintrag im DataFrame und wenden Sie das Modell an
for index, row in df.iterrows():
    text = row['text']
    text = preprocess(text)  # Stellen Sie sicher, dass Sie Ihre Preprocessing-Funktion hier verwenden
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    softmax_scores = softmax(scores)

    # Extrahieren Sie das vorhergesagte Sentiment
    predicted_label_id = np.argmax(softmax_scores)
    #predicted_sentiment = model.config.id2label[predicted_label_id]

    # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
    predicted_sentiments.append(predicted_label_id)
    actual_labels.append(row['label'])

# Fügen Sie die vorhergesagten Sentiments und die tatsächlichen Labels als neue Spalten zum DataFrame hinzu
df['predicted_sentiment'] = predicted_sentiments

# Vergleichen Sie die vorhergesagten Sentiments mit den tatsächlichen Labels
correct_predictions = (df['predicted_sentiment'] == df['label']).sum()
total_predictions = len(df)

# Berechnen Sie die Genauigkeit (Accuracy) der Vorhersagen
accuracy = correct_predictions / total_predictions

# Drucken Sie die Genauigkeit
print(f"Genauigkeit: {accuracy:.2%}")

Genauigkeit: 76.70%


## Test The Model on the test data

In [7]:
# Laden Sie Ihren CSV-Datensatz
csv_file = "../../data/test_data.csv"  # Ersetzen Sie durch den Pfad zu Ihrer CSV-Datei
df = pd.read_csv(csv_file)

# Leere Listen zum Speichern der vorhergesagten Sentiments und der tatsächlichen Labels
predicted_sentiments = []
actual_labels = []

# Durchlaufen Sie jeden Eintrag im DataFrame und wenden Sie das Modell an
for index, row in df.iterrows():
    text = row['text']
    text = preprocess(text)  # Stellen Sie sicher, dass Sie Ihre Preprocessing-Funktion hier verwenden
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    softmax_scores = softmax(scores)

    # Extrahieren Sie das vorhergesagte Sentiment
    predicted_label_id = np.argmax(softmax_scores)
    #predicted_sentiment = model.config.id2label[predicted_label_id]

    # Speichern Sie das vorhergesagte Sentiment und das tatsächliche Label
    predicted_sentiments.append(predicted_label_id)
    actual_labels.append(row['label'])

# Fügen Sie die vorhergesagten Sentiments und die tatsächlichen Labels als neue Spalten zum DataFrame hinzu
df['predicted_sentiment'] = predicted_sentiments

# Vergleichen Sie die vorhergesagten Sentiments mit den tatsächlichen Labels
correct_predictions = (df['predicted_sentiment'] == df['label']).sum()
total_predictions = len(df)

# Berechnen Sie die Genauigkeit (Accuracy) der Vorhersagen
accuracy = correct_predictions / total_predictions

# Drucken Sie die Genauigkeit
print(f"Genauigkeit: {accuracy:.2%}")

NameError: name 'tokenizer' is not defined

## Print out confusion matrix and statistics

In [28]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Berechnen Sie die Verwirrungsmatrix
conf_matrix = confusion_matrix(df['label'], df['predicted_sentiment'])

# Drucken Sie die Verwirrungsmatrix
print("Verwirrungsmatrix:")
print(conf_matrix)

# Berechnen Sie die Precision und den Recall für jede Klasse
precision = precision_score(df['label'], df['predicted_sentiment'], average=None)
recall = recall_score(df['label'], df['predicted_sentiment'], average=None)

# Drucken Sie die Precision und den Recall für jede Klasse
print("Precision pro Klasse:", precision)
print("Recall pro Klasse:", recall)

# Berechnen Sie den gewichteten Durchschnitt von Precision und Recall (Macro-Durchschnitt)
macro_precision = precision_score(df['label'], df['predicted_sentiment'], average='macro')
macro_recall = recall_score(df['label'], df['predicted_sentiment'], average='macro')

# Drucken Sie den gewichteten Durchschnitt von Precision und Recall
print(f"Macro Precision: {macro_precision:.2f}")
print(f"Macro Recall: {macro_recall:.2f}")


Verwirrungsmatrix:
[[247  58   7]
 [120 644 105]
 [ 17 150 652]]
Precision pro Klasse: [0.64322917 0.75586854 0.85340314]
Recall pro Klasse: [0.79166667 0.7410817  0.7960928 ]
Macro Precision: 0.75
Macro Recall: 0.78
